In [ ]:
from langchain.agents import create_agent  # import function to create agents
from langchain_openai import ChatOpenAI  # ChatOpenAI model wrapper
from dataclasses import dataclass  # dataclass decorator for simple data containers
from langchain.tools import tool, ToolRuntime  # tool decorator and runtime typing
import os  # access environment variables
import json  # JSON serialization/deserialization
from dotenv import load_dotenv  # helper to load .env files into environment
from langchain_tavily import TavilySearch  # Tavily search tool (external integration)
from langgraph.checkpoint.memory import InMemorySaver  # in-memory checkpointer for states
from langchain.agents.structured_output import ToolStrategy  # structured output strategy for agent responses
load_dotenv()  # load environment variables from a .env file if present

True

In [ ]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")  # read OpenAI API key from environment
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")  # read Langsmith API key from environment
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")  # read Tavily API key from environment

In [ ]:
model = ChatOpenAI(  # instantiate the ChatOpenAI model with desired settings
    model="gpt-5",  # model name to use
    temperature=0.1,  # low temperature for more deterministic outputs
    timeout=30,  # request timeout in seconds
)



In [ ]:
# SYSTEM_PROMPT: multi-line system instruction for the agent; do not modify the internal string content
SYSTEM_PROMPT = """You are an expert vegan/plant-based nutritionist and meal generator.
Your name is House of PlantAgent.

You have access to one tool, which you MUST use before generating each new recipe-generated response:

- tavily_search_tool: use this to search for a vegan recipe based on the user's stated preferences.

Rules:
- Think step-by-step.
- Based on your tool results, provide the ingredients, quantities, and cooking instructions in return to the user query.
- If the user asks non-recipe related questions, state that your purpose is only to generate user recipes.


"""

In [ ]:
tavily_search_tool = TavilySearch(  # create a TavilySearch instance with basic settings
        max_results=1,  # only keep the top result
        topic="general",  # general topic search
        search_depth = "advanced",  # depth parameter for the search
    )

# Wrap the TavilySearch instance so the agent gets a deterministic result shape.
@tool  # mark the following function as a tool the agent can call
def tavily_search_wrapper(query: str) -> str:
    """
    Call the TavilySearch tool and return a small JSON string containing:
    - title: top result title
    - url: top result URL
    - content: source text to ground the LLM
    Returning JSON avoids schema mismatches and makes the tool result easy to consume.
    """
    try:
        # tavily_search_tool is your TavilySearch instance created earlier.
        raw = tavily_search_tool.run(query)  # expected to return a list/dict of results
    except Exception as e:
        return json.dumps({"error": str(e)})  # return error as JSON string on failure

    # Inspect raw to find the first hit. Adjust keys to match Tavily's actual return.
    top = None  # placeholder for the top hit
    if isinstance(raw, list) and raw.get("results"):  # attempt to detect results in lists/dicts
        results = raw["results"]  # extract results field
        top = results[0] if results else raw  # take the first result if present
    elif isinstance(raw, dict) and raw:
        top = raw[0]  # try to get first element if dict-like (keep original behavior)
    else:
        top = raw  # fallback to raw if structure is unexpected

     # Extract canonical fields, with fallbacks to other likely keys
    if isinstance(top, dict):
        title = top.get("title") or top.get("name") or ""  # title heuristics
        url = top.get("url") or top.get("link") or ""  # url heuristics
        content = top.get("content") or top.get("snippet") or top.get("raw_content") or ""  # content heuristics
        score = top.get("score")  # optional score field
    else:
        title = str(top)  # fallback to stringifying top
        url = ""  # no url available
        content = str(top)  # use top as content
        score = None  # no score available

    # Return compact JSON so agent receives a consistent, usable tool result.
    return json.dumps(
        {"title": title, "url": url, "content": content, "score": score},
        ensure_ascii=False,
    )

In [ ]:
@dataclass  # simple dataclass to hold runtime context fields
class Context:
    """Custom runtime context schema."""
    user_id: str  # identifier for the user invoking the agent

In [ ]:
checkpointer = InMemorySaver()  # create an in-memory checkpointer for conversation state

In [ ]:
@dataclass  # response schema for structured agent outputs
class ResponseFormat:
    """Response schema for the agent."""
    # The response containing the title, ingredients, instructions.
    vegan_recipe: str  # the textual recipe and instructions
    # The URL from `tavily_search_wrapper`
    recipe_url: str  # source URL for the recipe

In [ ]:
agent = create_agent(  # create the agent with model, prompt, tools and formats
    model=model,  # the LLM instance to use
    system_prompt = SYSTEM_PROMPT,  # system-level instructions for the agent
    tools=[tavily_search_wrapper],  # list of callable tools exposed to the agent
    context_schema=Context,  # runtime context schema used when invoking the agent
    response_format=ToolStrategy(ResponseFormat),  # enforce structured output format
    checkpointer=checkpointer  # attach the checkpointer for state persistence
    

)

In [ ]:
user_input = "What's a quick tofu and noodles recipe?"  # sample user query to the agent

In [ ]:
    # `thread_id` is a unique identifier for a given conversation.
config = {"configurable": {"thread_id": "1"}}  # minimal config with a thread id

In [ ]:
for step in agent.stream(  # stream agent responses step-by-step
    {"messages": user_input},  # initial messages payload
    stream_mode="values",  # streaming mode to get incremental values
    config=config,  # pass the config with thread_id
    context=Context(user_id="1"),  # provide runtime context for this call
):
    step["messages"][-1].pretty_print()  # pretty-print the most recent message

================================ Human Message =================================

What's a quick tofu and noodles recipe?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_wrapper (call_wpm50WemMlA6L9TTuDGyQnTy)
 Call ID: call_wpm50WemMlA6L9TTuDGyQnTy
  Args:
    query: quick vegan tofu noodles recipe


KeyError: 0

Works but it doesn't look like the final generation meaningfully uses the tool's output...